<a href="https://colab.research.google.com/github/Flychuban/Space-Soup/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import requests
from transformers import pipeline
from bs4 import BeautifulSoup
import re

In [ ]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [ ]:
def search_urls_space_news():
  URL = "https://www.google.com/search?q=nasa+news&tbm=nws"
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [ ]:
hrefs = search_urls_space_news()

In [ ]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [ ]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [ ]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.nasa.gov/press-release/nasa-s-first-asteroid-sample-has-landed-now-secure-in-clean-room',
 'https://www.aljazeera.com/gallery/2023/9/25/major-asteroid-sample-brought-to-earth-in-nasa-first',
 'https://www.cnn.com/2023/09/25/world/brian-may-osiris-rex-asteroid-sample-bennu-scn-intl-scli/index.html',
 'https://www.npr.org/2023/09/24/1201386042/watch-live-nasa-sends-an-asteroid-sample-back-to-earth',
 'https://abcnews.go.com/US/nasas-osiris-rex-asteroid-sample-return-mission/story%3Fid%3D103408191',
 'https://www.google.com/search?q%3Dnasa%2Bnews%26tbm%3Dnws%26pccc%3D1',
 'https://www.nbcnews.com/science/space/first-nasa-returns-asteroid-samples-earth-rcna111474',
 'https://www.foxnews.com/us/nasa-reveals-latest-weapon-search-heavens-ufos-aliens',
 'https://www.wsj.com/video/watch-nasa-osiris-rex-mission-brings-home-samples-from-asteroid-bennu/E8068163-F481-4346-BEA3-EED27C577BE6.html',
 'https://www.space.com/nasa-osiris-rex-success-recovery-asteroid-sample',
 'https://www.

In [ ]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [ ]:
titles, articles = get_text_from_urls(cleaned_urls)

In [96]:
articles[2]

'\n\n      Nell Greenfieldboyce\n    \n \n                The sample return capsule from NASA\'s OSIRIS-REx mission is seen shortly after touching down in the desert at the Department of Defense\'s Utah Test and Training Range.\n                \n                    \n                    Keegan Barber/NASA\n                    \n                \nhide caption\n The sample return capsule from NASA\'s OSIRIS-REx mission is seen shortly after touching down in the desert at the Department of Defense\'s Utah Test and Training Range. Scientists are exulting over the safe arrival of a canister containing about a cup\'s worth of asteroid rocks, collected 200 million miles away, that landed in a Utah desert after a 7-year NASA mission sent to retrieve them.  The black pebbles and dirt are older than Earth, and are undisturbed remnants of the solar system\'s early days of planet formation. As part of an asteroid named Bennu, these rocks traveled unsullied through space for eons.  While bits of a

Split text from the article into chunks

In [ ]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))
sentences[-1]

['Subscribe to Here’s the Deal, our politics\n                 newsletter for analysis you won’t find anywhere\xa0else.',
 ' Thank you.',
 ' Please check your inbox to confirm.',
 ' \n\nMarcia Dunn, Associated Press\n\n\n                    Marcia Dunn, Associated Press\n                \n Leave your feedback NASA’s first asteroid samples fetched from deep space parachuted into the Utah desert Sunday to cap a seven-year journey.',
 ' In a flyby of Earth,\xa0the OSIRIS-REx spacecraft released the sample capsule from 63,000 miles (100,000 kilometers) out.',
 ' The small capsule landed four hours later on a remote expanse of military land, as the mothership set off after another asteroid.',
 ' Watch NASA’s post-landing news conference in the player above.',
 ' “We have touchdown!',
 '” Mission Recovery Operations announced, immediately repeating the news since the landing occurred three minutes early.',
 ' Officials later said the orange striped parachute opened four times higher than ant

In [ ]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[i][current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0
0
0


In [ ]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [93]:
joined_chunks[5]

["A space capsule from NASA’s Osiris-REx mission to the asteroid Bennu landed in Utah on Sunday.  The sample of rocks and dust—the largest ever collected from an asteroid—could help scientists better understand our solar system.  Photo: NASA/EPA/Shutterstock The union representing 146,000 auto workers called for a targeted strike after labor negotiations failed to reach a deal.  Here’s how this historic strike could affect the global car industry and what it could cost the U. S.  economy.  Photo: Paul Sancya/Associated Press Journal Editorial Report: The week's best and worst from Kim Strassel, Mene Ukueberuwa and Kyle Peterson.  Images: AP/Getty Images/Zuma Press Composite: Mark Kelly Utah Sen.  Mitt Romney announced he won’t seek re-election Wednesday.  He cited his age and the need for new leaders as reasons for his decision.  Photo: Shawn Thew/Shutterstock First came online shopping, then Prime, followed by AWS.  But when it comes to Amazon’s next big hit, the tech giant continues 

Summarize all chunks

In [ ]:
def summarize_chunks(chunks):
  summarized_texts = []
  for article_chunks in chunks:
    article_text = []
    result = summarizer(article_chunks, max_length=120, min_length=30, do_sample=False)
    article_text = ' '.join([summ['summary_text'] for summ in result])
    summarized_texts.append(article_text)
  return summarized_texts

In [ ]:
summarized_texts = summarize_chunks(joined_chunks)

Your max_length is set to 120, but your input_length is only 28. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 120, but your input_length is only 31. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)


In [98]:
summarized_texts[0]

' OSIRIS-REx probe lands in the desert in the US state of Utah . Scientists have high hopes for the sample, which landed on Sunday, saying it will provide a better understanding of the formation of our solar system and how Earth became habitable . The 6.21-billion-km journey marked the United States’ first sample return mission of its kind .'